In [2]:
#!/bin/python3
import ctypes
from ctypes import *
import numpy as np
import time
import cv2 as cv
from matplotlib import pyplot as plt
import SocketTransfer
from libqhy import *
qhyccd = CDLL('/usr/local/lib/libqhyccd.so')
qhyccd.GetQHYCCDParam.restype=c_double
qhyccd.OpenQHYCCD.restype=ctypes.POINTER(c_uint32)
# ref: https://www.qhyccd.com/bbs/index.php?topic=6356.0
# H.F. @20191115

Python Knowledge
+ https://docs.python.org/3/library/ctypes.html

QHY POST
+ https://www.qhyccd.com/bbs/index.php?topic=6316.0
+ https://www.qhyccd.com/bbs/index.php?topic=6356.0
+ https://www.qhyccd.com/bbs/index.php?topic=6663.0
+ https://github.com/EastEriq/QHYccd-matlab
+ https://note.youdao.com/coshare/index.html?token=AD62E2033F534E7CB3607CEC405CB3B3&gid=7195236?auto

In [3]:
# Create server object
server = SocketTransfer.socket_sender()

In [4]:
server.accept()

Waiting receive program
No receive program found!
Please reconnect after opening reveive program


In [4]:
ret = -1
qhyccd.InitQHYCCDResource()
qhyccd.ScanQHYCCD()
type_char_array_32 = c_char*32
id=type_char_array_32()
qhyccd.GetQHYCCDId(c_int(0), id)    # open the first camera
print(id.value)
cam = qhyccd.OpenQHYCCD(id)
qhyccd.SetQHYCCDStreamMode(cam, 1)  # 0 for single frame
qhyccd.InitQHYCCD(cam)

# Get Camera Parameters
chipw = c_double()
chiph = c_double()
w = c_uint()
h = c_uint()
pixelw = c_double()
pixelh = c_double() 
bpp = c_uint()
channels = c_uint32(1)
qhyccd.GetQHYCCDChipInfo(cam, byref(chipw), byref(chiph), byref(w), byref(h),
                         byref(pixelw), byref(pixelh), byref(bpp))

b'QHY163M-8cf4dc43a8fe4f8f'


0

## Capture Live Model

In [6]:
qhyccd.SetQHYCCDParam(cam, CONTROL_ID.CONTROL_USBTRAFFIC, c_double(60)) # don't work
qhyccd.SetQHYCCDParam(cam, CONTROL_ID.CONTROL_TRANSFERBIT, c_double(16))
qhyccd.SetQHYCCDParam(cam, CONTROL_ID.CONTROL_MANULPWM, c_double(255))
qhyccd.SetQHYCCDParam(cam, CONTROL_ID.CONTROL_COOLER, c_double(-15))
qhyccd.GetQHYCCDParam(cam, CONTROL_ID.CONTROL_CURTEMP)

28.6

In [7]:
qhyccd.SetQHYCCDParam(cam, CONTROL_ID.CONTROL_USBTRAFFIC, c_double(10)) 

0

In [8]:
qhyccd.GetQHYCCDParam(cam, CONTROL_ID.CONTROL_USBTRAFFIC) # don't work

10.0

In [9]:
qhyccd.GetQHYCCDParam(cam, CONTROL_ID.CONTROL_CURTEMP)

28.8

In [13]:
qhyccd.StopQHYCCDLive(cam)

0

In [15]:
exposure = c_double(0.001 * 10e6) # us
interval = 1 #.4s #2.5 fps
duration = 1*60 #s
time_start = time.time()
time_last = time.time()
time_now = time.time()
counter = 0
roi_w = c_uint(3522)
roi_h = c_uint(3522)
qhyccd.SetQHYCCDResolution(cam, 0, 0, roi_h, roi_w)
qhyccd.SetQHYCCDParam(cam, CONTROL_ID.CONTROL_EXPOSURE, exposure) # unit: us
imgdata = (ctypes.c_uint16 * roi_h.value * roi_w.value)()

qhyccd.BeginQHYCCDLive(cam)
while ( time_now - time_start < duration + interval ): # add one more interval
    while (time_now - time_last > interval) :
        qhyccd.GetQHYCCDLiveFrame(
            cam, byref(roi_h), byref(roi_w), byref(bpp), byref(channels), imgdata)
        img = np.asarray(imgdata).reshape([roi_w.value, roi_h.value])
        server.send_img(img)
        #cv.imwrite("%05d.tiff"%counter, img)
        print(counter, end=" ")
        counter += 1
        time_last = time_now
    time_now = time.time()
qhyccd.StopQHYCCDLive(cam)

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 

0

In [ ]:
exposure = c_double(0.001 * 10e6) # us
interval = 0.3 #.4s #2.5 fps
duration = 1*60 #s
time_start = time.time()
time_last = time.time()
time_now = time.time()
counter = 0
roi_w = c_uint(3522)
roi_h = c_uint(3522)
qhyccd.SetQHYCCDResolution(cam, 0, 0, roi_h, roi_w)
qhyccd.SetQHYCCDParam(cam, CONTROL_ID.CONTROL_EXPOSURE, exposure) # unit: us
imgdata = (ctypes.c_uint16 * roi_h.value * roi_w.value)()

test = np.uint16(np.random.normal(size=(3522, 3522)))

qhyccd.BeginQHYCCDLive(cam)
while ( time_now - time_start < duration + interval ): # add one more interval
    while (time_now - time_last > interval) :
        #hyccd.GetQHYCCDLiveFrame(
        #   cam, byref(roi_h), byref(roi_w), byref(bpp), byref(channels), imgdata)
        #mg = np.asarray(imgdata).reshape([roi_w.value, roi_h.value])
        #server.send_img(img)
        server.send_img(test)
        #cv.imwrite("%05d.tiff"%counter, img)
        print(counter, end=" ")
        counter += 1
        time_last = time_now
    time_now = time.time()
qhyccd.StopQHYCCDLive(cam)

In [25]:
qhyccd.CloseQHYCCD(cam)
qhyccd.ReleaseQHYCCDResource()

0